In [2]:
import pandas as pd
import numpy as np

import os
import unicodedata
import re
import json

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

from prepare import prep_create_labels
from preprocess import prep_job_data, split_job_data, add_columns

# Wrangle the Data
- use functions from previous work to get a working train dataset

In [3]:
df = pd.read_json('indeed-data-jobs-FINAL.json')
df = prep_create_labels(df).reset_index(drop=True)
df = prep_job_data(df, 'job_description', extra_words=['job', 'description']).reset_index(drop=True)
df = add_columns(df)
df.sample(5)

/Users/brandmarz/data-science-projects/nlp-data-job-classifier/preprocess.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  .apply(lemmatize)
/Users/brandmarz/data-science-projects/nlp-data-job-classifier/preprocess.py:124: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['stemmed'] = df[column].apply(basic_clean).apply(stem)
/Users/brandmarz/data-science-projects/nlp-data-job-classifier/preprocess.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

,job_title,company,location,is_remote,salary,post_date,date_accessed,job_description,label,clean,stemmed,lemmatized,words,doc_length
154,"Data Scientist, Analytics",Castlight Health,"San Francisco, CA 94107",0,,9 days ago,2021-03-05,"Data Scientist, Analytics At Castlight Health,...",DS,data scientist analytics castlight health miss...,data scientist analyt at castlight health our ...,data scientist analytics at castlight health o...,"[data, scientist, analytics, castlight, health...",224
253,Data Analyst,CooperSurgical,"Boston, MA",0,,3 days ago,2021-03-05,We are looking for a Data Analyst to join Coop...,DA,looking data analyst join coopersurgical help ...,we are look for a data analyst to join coopers...,we are looking for a data analyst to join coop...,"[looking, data, analyst, join, coopersurgical,...",305
635,Machine Learning Engineer - Style Discovery,Stitch Fix,"San Francisco, CA",0,,30+ days ago,2021-03-05,"About the Team At Stitch Fix, we're transform...",MLE,team stitch fix transforming way people find l...,about the team at stitch fix were transform th...,about the team at stitch fix were transforming...,"[team, stitch, fix, transforming, way, people,...",385
246,Data Analyst,Kaizen Technologies,"Edison, NJ 08820",0,,1 day ago,2021-03-05,"Required Skills : TSQL, SQL Job Description :...",DA,required skill tsql sql ability write transact...,requir skill tsql sql job descript abil to wri...,required skill tsql sql job description abilit...,"[required, skill, tsql, sql, ability, write, t...",59
406,Data Engineer,DGN Technologies,Remote,1,Up to $60 an hour,22 days ago,2021-03-05,"Description: Data Engineer: Designs, builds an...",DE,data engineer design build oversees deployment...,descript data engin design build and overse th...,description data engineer design build and ove...,"[data, engineer, design, build, oversees, depl...",255


In [4]:
train, validate, test = split_job_data(df)
print(f'train: {round(train.shape[0]/len(df),2)}')
print(f'validate: {round(validate.shape[0]/len(df),2)}')
print(f'test: {round(test.shape[0]/len(df),2)}')

train: 0.6
validate: 0.2
test: 0.2


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 426 entries, 420 to 531
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   label       426 non-null    object
 1   job_title   426 non-null    object
 2   company     426 non-null    object
 3   location    426 non-null    object
 4   is_remote   426 non-null    int64 
 5   clean       426 non-null    object
 6   words       426 non-null    object
 7   doc_length  426 non-null    int64 
dtypes: int64(2), object(6)
memory usage: 30.0+ KB


In [5]:
train.head()

,label,job_title,company,location,is_remote,clean,words,doc_length
420,DE,Data Engineer,"Stefanini, Inc","Richmond, VA",0,stefanini looking data engineer richmond va re...,"[stefanini, looking, data, engineer, richmond,...",216
250,DA,Financial Data Analyst (vehicle retail Domain),SILVERLINK TECHNOLOGIES LLC,"Bowling Green, KY",0,hiplease go let know interestjob title financi...,"[hiplease, go, let, know, interestjob, title, ...",64
426,DE,Data Engineer I or II - Can Be Remote Based On...,Associated Bank,"Milwaukee, WI",1,associated bank equal opportunity employer com...,"[associated, bank, equal, opportunity, employe...",366
170,DS,Associate Data Scientist,Gap Inc.,United States,0,gap inc brand bridge gap see world old navy de...,"[gap, inc, brand, bridge, gap, see, world, old...",337
581,MLE,Automation/Artificial Intelligence Machine Lea...,LOCKHEED MARTIN CORPORATION,"Littleton, CO 80125",0,coolest job planet lockheed martin space lockh...,"[coolest, job, planet, lockheed, martin, space...",499
